In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']
df = pd.read_csv(url, names=columns)

X = df.drop('Outcome', axis=1)
y = df['Outcome']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

X_train = torch.tensor(X_train, dtype=torch.float32).view(-1, 1, 8)  # Reshape for CNN: (batch_size, channels, features)
X_test = torch.tensor(X_test, dtype=torch.float32).view(-1, 1, 8)    # Reshape for CNN: (batch_size, channels, features)
y_train = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

In [2]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()

        # First convolutional layer: Input is (batch_size, 1, 8), output will be (batch_size, 32, 8)
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=3, padding=1)

        # Second convolutional layer: Input is (batch_size, 32, 8), output will be (batch_size, 64, 8)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1)

        # Pooling layer: reduce the size of the feature map
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)

        # Fully connected layers after the convolutional layers
        self.fc1 = nn.Linear(64 * 4, 128)  # Flattened dimension after pooling
        self.fc2 = nn.Linear(128, 1)       # Output layer for binary classification

        self.sigmoid = nn.Sigmoid()         # Sigmoid activation for binary classification

    def forward(self, x):
        # Apply convolution + ReLU activation
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))

        # Pooling layer
        x = self.pool(x)

        # Flatten the feature map
        x = x.view(-1, 64 * 4)  # Flatten the output of the convolutional layers

        # Fully connected layers
        x = torch.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))  # Output layer

        return x


In [3]:
# Initialize the model, loss function, and optimizer
model = CNNModel()
criterion = nn.BCELoss()  # Binary cross-entropy loss for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 1500
for epoch in range(epochs):
    model.train()  # Set the model to training mode
    optimizer.zero_grad()  # Zero the gradients

    # Forward pass
    output = model(X_train)

    # Compute the loss
    loss = criterion(output, y_train)

    # Backward pass
    loss.backward()

    # Update the weights
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")


Epoch [10/1500], Loss: 0.5694
Epoch [20/1500], Loss: 0.4731
Epoch [30/1500], Loss: 0.4515
Epoch [40/1500], Loss: 0.4335
Epoch [50/1500], Loss: 0.4169
Epoch [60/1500], Loss: 0.3998
Epoch [70/1500], Loss: 0.3812
Epoch [80/1500], Loss: 0.3604
Epoch [90/1500], Loss: 0.3405
Epoch [100/1500], Loss: 0.3203
Epoch [110/1500], Loss: 0.3023
Epoch [120/1500], Loss: 0.2832
Epoch [130/1500], Loss: 0.2680
Epoch [140/1500], Loss: 0.2504
Epoch [150/1500], Loss: 0.2324
Epoch [160/1500], Loss: 0.2155
Epoch [170/1500], Loss: 0.1993
Epoch [180/1500], Loss: 0.1821
Epoch [190/1500], Loss: 0.1680
Epoch [200/1500], Loss: 0.1523
Epoch [210/1500], Loss: 0.1361
Epoch [220/1500], Loss: 0.1224
Epoch [230/1500], Loss: 0.1099
Epoch [240/1500], Loss: 0.0980
Epoch [250/1500], Loss: 0.0883
Epoch [260/1500], Loss: 0.0788
Epoch [270/1500], Loss: 0.0704
Epoch [280/1500], Loss: 0.0626
Epoch [290/1500], Loss: 0.0556
Epoch [300/1500], Loss: 0.0489
Epoch [310/1500], Loss: 0.0433
Epoch [320/1500], Loss: 0.0384
Epoch [330/1500],

In [4]:
# Evaluate the model on the test set
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Disable gradient computation for inference
    predictions = model(X_test)
    predicted_labels = (predictions > 0.5).float()  # Convert to 0 or 1 based on threshold

    # Calculate accuracy
    accuracy = accuracy_score(y_test, predicted_labels)
    print(f"Model accuracy on test set: {accuracy * 100:.2f}%")

Model accuracy on test set: 67.53%
